In [2]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-mxzb7cva
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-mxzb7cva
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=d531b87ed4ab5fb74c14073c46081eecb3b3b610bd3f98971c0671fb3c00a42a
  Stored in directory: /tmp/pip-ephem-wheel-cache-j06d6lkl/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

// ядро
__global__ void kernel(double* input, double* output, int n) {
    // выделение общей памяти в блоке
    extern __shared__ double sdata[];
    // получение текущего потока в блоке и размерность блока
    unsigned int tid = threadIdx.x;
    unsigned int bdim = blockDim.x;

    double sum=0;
    // каждый поток складывает значения векторов, начиная со значения, чей индекс равен номеру потока
    // и с шагом равным общему числу потоков
    for(int i = blockIdx.x * blockDim.x + threadIdx.x; i < n; i += blockDim.x * gridDim.x){
        sum += input[i];
    }
    // сохраняется полученное значение как локальная сумма потока в блоке
    sdata[tid] = sum;
    __syncthreads();

    int step = blockDim.x >> 1;

    while(tid + step < bdim && step > 0  ){
        sdata[tid] += sdata[tid+step];
        bdim = step;
        step = step >> 1;
        __syncthreads();
    }
    // сохранение полученного значения в блоке
    output[blockIdx.x] = sdata[0];
}


// метод подсчета размерностей грида и блока
void count_dims(int& blocksPerGrid, int& threadsPerBlock, int n){
    int pow = floor(log2((float)n/2));
    // оптимальное число потоков для операций над вектором n/2
    int numberOfThreads = 1<<pow;
    // в зависимости от числа потоков установка числа потоков в блоке
    threadsPerBlock = numberOfThreads < 1024 ? numberOfThreads : 1024;
    // в зависимости от числа потоков и числа потоков в блоке установка числа блоков в гриде
    blocksPerGrid = numberOfThreads < 1024 ? 1 : numberOfThreads / threadsPerBlock;
}

// функция суммы элементов вектора на GPU
double execute_gpu(double* inArray, double* outArray, int n){
    // подсчет числа блоков в гриде и числа потоков в блоке
    int blocksPerGrid;
    int threadsPerBlock;
    count_dims(blocksPerGrid, threadsPerBlock, n);


     // Выделение памяти на устройстве для копирования самого вектора
    double* indev = NULL;
    cudaError_t cuerr = cudaMalloc((void**)&indev, n * sizeof(double));
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot allocate device array for indev: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Выделение памяти на устройстве для промежуточных сумм блоков
    double* outdev = NULL;
    cuerr = cudaMalloc((void**)&outdev, blocksPerGrid*threadsPerBlock * sizeof(double));
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot allocate device array for outdev: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cuerr = cudaEventCreate(&start);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot create CUDA start event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    cuerr = cudaEventCreate(&stop);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot create CUDA end event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Копирование данных вектора с хоста на девайс
    cuerr = cudaMemcpy(indev, inArray, n * sizeof(double), cudaMemcpyHostToDevice);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot copy a array from host to device: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Установка точки старта
    cuerr = cudaEventRecord(start, 0);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot record CUDA event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    //Запуск ядра
    while(n>1){
        kernel <<< blocksPerGrid, threadsPerBlock, threadsPerBlock*sizeof(double) >>> (indev, outdev,n);
        cuerr = cudaGetLastError();
        if (cuerr != cudaSuccess){
            fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
                cudaGetErrorString(cuerr));
            return 0;
        }
        n = blocksPerGrid;
        printf("blgr %d", n);
        count_dims(blocksPerGrid, threadsPerBlock, n);
        cudaFree(indev);
        indev=outdev;
        outdev=NULL;
        cudaError_t cuerr = cudaMalloc((void**)&outdev, sizeof(double)*n);
        if(cuerr!=cudaSuccess){
            fprintf(stderr, "Cannot allocate device array for outdev %s\n", cudaGetErrorString(cuerr));
        }
    }

    // Синхронизация устройств
    cuerr = cudaDeviceSynchronize();
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot synchronize CUDA kernel: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Установка точки окончания
    cuerr = cudaEventRecord(stop, 0);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Копирование результата с девайса на хост
    cuerr = cudaMemcpy(outArray, indev, sizeof(double), cudaMemcpyDeviceToHost);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // подсчет времени
    cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
    double time = gpuTime/ 1000;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(indev);
    cudaFree(outdev);
    return time;
}


// функция суммы элементов вектора на cpu
double execute_cpu(double* inp, double* outp, int n){
  double time = clock();
  double sum=0;
  for (int i=0; i<n; ++i){
      sum+=inp[i];
  }
  outp[0]=sum;
  //printf("CPU %f\n", outp[0]);
  double endTime= clock()-time;
  return endTime/=CLOCKS_PER_SEC;
}



// основная функция
int main(int argc, char* argv[])
{
    // размерность матриц
    int n = 8192;

    // Выделение памяти на хосте
    double* input = (double*)malloc(sizeof(double)*n);
    double* output_cpu=(double*)malloc(sizeof(double));
    double* output_gpu=(double*)malloc(sizeof(double));


    // заполнение матриц
    for (int i = 0; i < n; i++) {
        input[i] = 1;
    }
    double time_cpu=0;
    double time_gpu=0;
    // цикл для усреднения значений
    for(int k=0;k<32;++k){
        // вызов суммы элементов вектора на CPU
        time_cpu += execute_cpu(input, output_cpu, n);
        // вызов суммы элементов вектора на GPU
        time_gpu += execute_gpu(input,  output_gpu, n);

    }
    printf("SUM VECTOR'S ELEMENTS ON CPU: %f;\nTIME_CPU: %f\n", output_cpu[0], time_cpu/32);
    printf("SUM VECTOR'S ELEMENTS ON GPU: %f;\nTIME_GPU: %f\n", output_gpu[0], time_gpu/32);
    // подсчет ускорения
    printf("A: %f\n", time_cpu/time_gpu);
    // освобождение выделенной памяти
    free(input);
    free(output_cpu);
    free(output_gpu);

    return 0;
}


blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1blgr 4blgr 1SUM VECTOR'S ELEMENTS ON CPU: 8192.000000;
TIME_CPU: 0.000029
SUM VECTOR'S ELEMENTS ON GPU: 8192.000000;
TIME_GPU: 0.000050
A: 0.584518

